# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
import numpy as np
import ufl

from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem
from dolfinx.mesh import CellType, MeshTags, create_box, locate_entities_boundary
L = 20.0
mesh = create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, 1, 1]], [20, 5, 5], CellType.hexahedron)
V = fem.VectorFunctionSpace(mesh, ("CG", 2))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

left_facets = locate_entities_boundary(mesh, mesh.topology.dim - 1, left)
right_facets = locate_entities_boundary(mesh, mesh.topology.dim - 1, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full(len(left_facets), 1, dtype=np.int32), np.full(len(right_facets), 2, dtype=np.int32)])
sorted_facets = np.argsort(marked_facets)
facet_tag = MeshTags(mesh, mesh.topology.dim-1, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [9]:
u_bc = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [10]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
bcs = [fem.DirichletBC(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`). 

In [11]:
B = fem.Constant(mesh, PETSc.ScalarType((0, 0, 0)))
T = fem.Constant(mesh, PETSc.ScalarType((0, 0, 0)))

Define the test and solution functions on the space $V$

In [12]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [13]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [14]:
# Elasticity parameters
E = PETSc.ScalarType(1.0e4)
nu = PETSc.ScalarType(0.3)
mu = fem.Constant(mesh, E/(2*(1 + nu)))
lmbda = fem.Constant(mesh, E*nu/((1 + nu)*(1 - 2*nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [15]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [16]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [17]:
problem = fem.NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [18]:
from dolfinx.nls import NewtonSolver
solver = NewtonSolver(MPI.COMM_WORLD, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

We create a function to plot the solution at each time step.

In [19]:
import pyvista
from dolfinx.plot import create_vtk_topology
# Activate framebuffer, required for visualziation
pyvista.start_xvfb(wait=0.05)
# Create background plotter and mesh to attach values to
topology, cell_types = create_vtk_topology(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)
def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter(window_size=[960,480])
    # Add time-stamp
    p.add_text(f"Time: {t}", font_size=12, name="timelabel")
    p.add_text("Deformed configuration", name="title", position="upper_edge")
   
    # Update point values on pyvista grid

    topology, cell_types = create_vtk_topology(V)
     # We create a geometry for our modified mesh using the dof coordinates
    geometry = V.tabulate_dof_coordinates()
    # As we are dealing with a vector field, we reshape the underlying dof array to accommedate for the three dimensional space
    num_dofs = V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts
    values = np.zeros((num_dofs, 3), dtype=np.float64)
    values[:, :mesh.geometry.dim] = uh.x.array.real.reshape(num_dofs, V.dofmap.index_map_bs)

    # Create grid defined by the function space for visualization
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    function_grid["u"] = values
    function_grid.set_active_vectors("u")
    # Warp mesh by deformation
    warped = function_grid.warp_by_vector("u", factor=1)
    
    # Add mesh to plotter and visualize
    actor = p.add_mesh(warped)
    p.show_axes()
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)

plot_function(0, u)

  warnings.warn("Plotting of higher order functions is experimental.")



ViewInteractiveWidget(height=480, layout=Layout(height='auto', width='100%'), width=960)

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [ ]:
from dolfinx.log import set_log_level, LogLevel
set_log_level(LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert(converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    plot_function(n, u)

2021-12-25 17:07:25.161 ( 168.262s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:25.927 ( 169.029s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:26.333 ( 169.434s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2021-12-25 17:07:26.526 ( 169.628s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:26.993 ( 170.095s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2021-12-25 17:07:27.241 ( 170.343s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:27.695 ( 170.797s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


ViewInteractiveWidget(height=480, layout=Layout(height='auto', width='100%'), width=960)

2021-12-25 17:07:31.253 ( 174.355s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:31.959 ( 175.061s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:32.400 ( 175.501s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2021-12-25 17:07:32.668 ( 175.770s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:33.185 ( 176.286s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2021-12-25 17:07:33.457 ( 176.559s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:33.990 ( 177.092s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


ViewInteractiveWidget(height=480, layout=Layout(height='auto', width='100%'), width=960)

2021-12-25 17:07:38.428 ( 181.530s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:39.075 ( 182.177s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:39.533 ( 182.634s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2021-12-25 17:07:39.795 ( 182.897s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:40.318 ( 183.420s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 5.33026 (tol = 1e-08) r (rel) = 0.0472992(tol = 1e-08)
2021-12-25 17:07:40.566 ( 183.668s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2021-12-25 17:07:41.042 ( 184.143s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration